In [ ]:
import warnings
import numpy as np
import pandas as pd

from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, StackingClassifier, BaggingRegressor, \
    GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
warnings.filterwarnings('ignore')

In [ ]:
parameters_ensemble = {'n_estimators': np.arange(20,101,20),
                       'max_features': np.arange(3,24,10)}

# Классификация

In [ ]:
def print_classification_model_metrics(estimator, y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(estimator.score(X_test, y_test))

In [ ]:
phone_df = pd.read_csv("../data/Smartphone_chosse_preprocessed.csv")
phone_df

In [ ]:
y = phone_df["blue"]
y

In [ ]:
X = phone_df[phone_df.columns[2:phone_df.shape[1]]]
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Базовая модель DTC

In [ ]:
%%time
parameters_dtc = {'max_depth': np.arange(5,16,1)}
dtc = DecisionTreeClassifier()
dtc_base = GridSearchCV(dtc, parameters_dtc).fit(X_train, y_train)
dtc_base.best_params_

In [ ]:
print_classification_model_metrics(dtc_base, y_test, dtc_base.predict(X_test))

# Модель BaggingClassifier

In [ ]:
%%time
bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=dtc_base.best_params_['max_depth']))
model = GridSearchCV(bag, parameters_ensemble).fit(X_train, y_train)
model.best_params_

In [ ]:
print_classification_model_metrics(model, y_test, model.predict(X_test))

# Модель GradientBoostingClassifier

In [ ]:
%%time
gbc = GradientBoostingClassifier()
model = GridSearchCV(gbc, parameters_ensemble).fit(X_train, y_train)
model.best_params_

In [ ]:
print_classification_model_metrics(model, y_test, model.predict(X_test))

# Модель StackingClassifier

Сделаем стекинг из двух композиций и применим DecisionTreeClassifier с найденными гиперпараметрами

In [ ]:
%%time
model = StackingClassifier(estimators=[('bag',bag), ('gbc',gbc)],
                           final_estimator=dtc_base).fit(X_train, y_train)

In [ ]:
print_classification_model_metrics(model, y_test, model.predict(X_test))
